# Monte Carlo Parte 3


Nesta parte tentaremos resolver o problema de calcular a energia de um sistema atômico multidimensional. Lembrando que nas partes 1 e 2 resolvemos problemas considerando o sistema como unidimensional e verificamos que a solução de integrais unidimensionais como o método Monte Carlo não é tão vantajoso quando comparado com métodos numéricos clássicos, tais como o método do trapézio composto.

- O problema que devemos resolver é:

   Considere um sistema formado por 100 átomos de Argônio, em uma temperatura $T = 90 K$, pressão $p = 20 atm$ e densidade d = 1,394 $g/cm³$. A partir do potencial de interação de LJ e utilizando uma distribuição uniforme. Calcule a energia interna média deste sistema.
   
Para resolver esse problema vamos considerá-lo em partes.

###### Primeira Parte

Vamos estabelecer que os 100 átomos de Argônio estão limitados em uma região compreendida por uma caixa de dimensões cúbicas. Assim deveremos calcular os lados desta caixa a partir da quantidade de átomos que estamos colocando na caixa e sua densidade.

As relações usadas para o tamanho dos lados da caixa são:

1) calcular a quantidade de matéria (n) relacionada com N = 100 átomos.

$$\frac{1 mol (Ar)}{6,02 \times 10^{23} átomos} = \frac{n (Ar)}{100 átomos}$$

$$n = 1,661 \times 10^{-22} mol (Ar)$$

2) calcular a massa (m) relacionada com a quantidade de matéria (n) através da massa Molar do argônio $MM(Ar) = 39,963 \frac{g}{mol}$.

$$\frac{39,963g}{1 mol (Ar)} = \frac{m}{1,661 \times 10^{-22} mol (Ar)}$$

$$m = 6,6383 \times 10^{-21}g$$

3) Como o volume de uma caixa cúbica é $v = L^3$ e a densidade é $d = m/v$, então o lado da caixa é:

$$L = \Big(\frac{m}{d}\Big)^{\frac{1}{3}}.$$

4) Substituindo o valor da massa e a densidade do Argônio, teremos: 

$$L = 1,682 \times 10^{-7} cm$$

5) Convertendo para angstrom temos:

$$L = 16,82 angstrom$$

Sabendo esses passos podemos criar uma função em python que calcula o lado da caixa de dimensões cúbicas.

In [53]:
def lado_cubo(N, MM, d):
    '''
    Essa função recebe como parâmetros:
    o número de átomos do sistema, a
    massa molar do átomo e a densidade.
    Retornando o lado da caixa de dimensões
    cúbicas.
    
    Unidades de Medida (entrada)
    ------------------
    número de átomos(N)   [sem unidades]
    Massa Molar(MM)       [g/mol]
    densidade(d)          [g/cm^3]
    
    Unidades de Medida (saida/retorno)
    ------------------
    lado(l)    [angstrom]
    '''
    
    Na = 6.02*(10**23)
    quantidade_materia = N / Na
    massa = MM * quantidade_materia
    
    lado = (massa/d)**(1/3)
    lado_metros = lado/100
    lado_angstrom = lado_metros * (10**10)
    
    return lado_angstrom
    

In [9]:
# Vamos passar as informações do problema e verificar
#se o função lado_cubo retorna o valor de 16,82 angstrom.

numeros_de_atomos = 100
massa_molar = 39.963
densidade = 1.394
print(lado_cubo(numeros_de_atomos, massa_molar, densidade))

16.824142545641326


Note que o valor de retorno da função lado_cubo() foi exatamente o que esperavamos, assim temos a primeira parte concluida :).

###### Segunda parte

Nesta parte vamos criar uma função que gere os valores das coordenadas x, y e z da partícula de maneira aleatoria  e retorne o valor da coordenada espacial $r$ que está contida no interior da caixa de dimensões cúbicas. Podemos descrever o valor de $r$ como sendo:

$$r = (x² + y² + z²)^{\frac{1}{2}}$$

In [54]:
import random


def coords_x_y_z(l):
    '''
    Essa função recebe como parâmetro
    apenas o valor do lado da caixa de
    dimensões cúbicas. Retornando a 
    posição x, y, z de uma partícula no
    interior da caixa.    
    '''
    
    minimo = -l/2
    maximo = l/2
    
    x = random.uniform(minimo, maximo)
    y = random.uniform(minimo, maximo)
    z = random.uniform(minimo, maximo)
    
    r = (x**2 + y**2 + z**2)**(1/2)
    
    return r


# Teste da função coords_x_y_z(l)

numeros_de_atomos = 100
massa_molar = 39.963
densidade = 1.394
lado = lado_cubo(numeros_de_atomos, massa_molar, densidade)

print(coords_x_y_z(lado))
    
    

6.183970924900797


Executando a função coords_x_y_z() percebemos que as coordenadas geradas estão de fato no interior da caixa.

###### Terceira Parte

Agora que já temos como calcular as coordenas das partículas no interior da caixa cúbica de forma aleatória podemos calcular o valor da interação de pares considerando o potencial de LJ. A estrutura funcional do potencial de LJ é

$$U(r) = 4 \epsilon \Biggl[\Biggl(\frac{\sigma}{(x² + y² + z²)^{\frac{1}{2}}}\Biggl)^{12}-\Biggl(\frac{\sigma}{(x² + y² + z²)^{\frac{1}{2}}}\Biggl)^{6} \Biggl]$$

Portanto deveremos implementar a função acima para calcular o potencial de interação.

In [57]:
def potencial(r, e, si):
    '''
    Essa função é o potencial de LJ, que 
    recebe como parâmetros de entrada as 
    posições aleatórias que foram geradas 
    a partir das dimensões de uma caixa 
    cúbica, a energia de interação epsilon
    e a distância de interação sigma.
    Retorna o potencial de interação.
    
    Unidades de Medida (entrada)
    ------------------
    distancia(r)          [angstrom]
    epsilon(e)            [kcal/mol]
    sigma(si)             [angstrom]
    
    Unidades de Medida (saida/retorno)
    ------------------
    potencial(U)          [kcal/mol]    
    '''
    
    U = 4*e*((si/r)**12 - (si/r)**6)
    return U

# Teste da função potencial(r, e, si)

numeros_de_atomos = 100
massa_molar = 39.963
densidade = 1.394
lado = lado_cubo(numeros_de_atomos, massa_molar, densidade)


r = coords_x_y_z(lado)
epsilon =  0.2378
sigma = 3.41


U = potencial(r, epsilon, sigma)
print(U)
    

-0.00047201029552427343


###### Quarta Parte

Agora devemos somar as energias potenciais correspondentes a todas as partículas que fazem parte do sistema de interesse. Neste caso sabemos que:

$$U(r) = \sum_{i}\sum_{j}U(r_{ij}) = \sum_{i}\sum_{j}U_{ij}(r).$$

In [70]:
# Somando as energias potenciais

numero_de_atomos = 100
massa_molar = 39.963
densidade = 1.394

lado = lado_cubo(numeros_de_atomos, massa_molar, densidade)

epsilon =  0.2378
sigma = 3.41

U_positivo = 0
U_total = 0
for i in range(numero_de_atomos):
    r = coords_x_y_z(lado)
    U = potencial(r, epsilon, sigma)
    U_total += U
    
    if U > 0:
        U_positivo += 1   
      
    print(U)


print('\n')    
print(f'Energias Positivas: {U_positivo} de {numero_de_atomos}')
print('------')    
print(f'Energia Total: {U_total}')    

-0.0007691642954086481
-0.003426733364795434
-0.002767804216393668
-0.0025559778579728586
-0.018049528001352455
-0.003515133694783241
-0.23273987819719646
-0.0047135237347985205
-0.004715538904015718
-0.0006161196252525886
-0.007954056566997778
-0.0045750927521950565
-0.050363162726443936
-0.0022647778299958714
-0.0007159586726082263
-0.008089617698907241
-0.0007931164754923088
-0.0019613735303906283
-0.0032977292074801495
-0.0035391370969864917
-0.0003197400846424295
-0.0056607307144717605
-0.003142314800791984
-0.0034842130423506966
-0.002586989557463371
-0.00076312988529643
-0.0029764393766113216
-0.0523612849917509
-0.0025861608240090794
-0.001304328887022319
-0.0008120435153779531
-0.21571295264174992
-0.00038823811045446253
-0.09853378723469178
-0.012159079677062338
-0.0028493074312453716
-0.0006815692790440448
-0.013023344088899375
-0.0018210401782076495
-0.0010597442153005604
-0.006707300425014059
-0.0006937125752245307
-0.003462056463165116
-0.008913914897021109
-0.00066345189

###### Quinta parte

Agora será necessário fazer uma discussão para calcular o valor média da energia total do sistema. Como sabemos a termodinâmica trabalha com valores médios. Portanto o valor médio de uma propriedade do sistema é dada por:

$$ \langle f \rangle = \sum_{i} f_{i}P_{i}.$$

Pela distribuição de Boltzmann, 

$$P_{i} = \frac{e^{-\beta E_i}}{\sum_{i}e^{-\beta E_i}}.$$

Logo o valor médio de uma propriedade é dado na forma:

$$\langle f \rangle = \frac{\sum_{i} f_{i} e^{-\beta E_i}}{\sum_{i}e^{-\beta E_i}}.$$

Como nosso sistema é composto por um enorme conjunto de partículas então a propriedade média discreta $\langle f \rangle$ pode ser substituida por uma propriedade contínua, assumindo que existem também uma coleção enorme de estados $E_i$ no sistema de interesse. Logo $E_i$ pode ser entendido como a energia total do sistema e sabemos que a energia total é descrita pela seguinte Hamiltoniana:

$$H(p,r) = k(p) + U(r),$$

em que a energia cinética é dada na forma $k(p) = \frac{p²}{2m}$, enquanto o energia potencial $U(r)$ pode assumir as mais diversas formas. Assumindo um forma contínua da propriedade $f$ teremos:

$$\langle f \rangle = \frac{\int_{p}\int_{r} f(p, r) e^{-\beta H(p,r)} dp dr}{\int_{p}\int_{r} e^{-\beta H(p,r)} dp dr}.$$

A equação acima descreve a forma de calcularmos o valor de uma propriedade média a partir da energia total do sistema.

No caso de buscarmos calcular a energia média do sistema devemos considerar que:

$$\langle E \rangle = - \frac{\partial ln Z}{\partial \beta},$$

sabendo que a função de partição $Z$ é da forma

$$Z = \int_{p}\int_{r} e^{-\beta H(p,r)} dp dr.$$

Expandindo a função de partição em termo da Hamiltoniana temos

$$Z = \int_{p}\int_{r} e^{-\beta (k(p) + U(r))} dp dr$$

$$Z = \int_{p} e^{-\beta k(p)} dp \int_{r} e^{-\beta U(r)} dr,$$ 

como sabemos o valor da função energia cinética, também sabemos a solução da integral que envolve somente as coordenadas da energia cinética,

$$\int_{p} e^{-\beta k(p)} dp = \Big(\frac{2m\pi}{\beta}\Big)^{\frac{3N}{2}}.$$

Portanto a função de partição é da forma:

$$Z = \Big(\frac{2m\pi}{\beta}\Big)^{\frac{3N}{2}} \int_{r} e^{-\beta U(r)} dr,$$

aplicando o logaritmo natural a função de partição teremos, 

$$ln(Z) = \frac{3N}{2} (2m\pi) - \frac{3N}{2} \beta + ln\Big(\int_{r} e^{-\beta U(r)} dr\Big).$$

Derivando e tomando e tomando o negativo da função de partição,

$$- \frac{\partial ln Z}{\partial \beta} = \frac{3N}{2} \frac{1}{\beta} + \frac{\int_{r} U(r) e^{-\beta U(r)} dr}{\int_{r} e^{-\beta U(r)} dr},$$

sendo $\beta = 1/k_b T$, então a energia média é:

$$ \langle E \rangle = \frac{3N}{2} k_b T + \frac{\int_{r} U(r) e^{-\beta U(r)} dr}{\int_{r} e^{-\beta U(r)} dr}.$$

E o termo que estamos interessados em calcular com a simulação é a integral:

$$ \frac{\int_{r} U(r) e^{-\beta U(r)} dr}{\int_{r} e^{-\beta U(r)} dr}.$$

Como o problema deseja que a solução seja a partir de uma distribuição uniforme, então temos:

$$ \frac{\langle U(\eta) e^{-\beta U(\eta)} \rangle_{n}}{\langle e^{-\beta U(\eta)} \rangle_{n}}$$

em que $\eta$ não as coordenadas aleatórias geradas.